In [1]:
%load_ext sql
%sql sqlite://

/Library/Python/2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Library/Python/2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


'Connected: None@None'

In [6]:
# Utility functions...
from IPython.core.display import display_html, HTML
def to_html_table(res, style=None):
    html = '<table' + (' style="' + style + '"' if style else '') + '><tr><th>'
    html += '</th><th>'.join(res.keys) + '</th></tr><tr><td>'
    html += '</td></tr><tr><td>'.join(['</td><td>'.join([str(cell) for cell in row]) for row in list(res)])
    return html + '</tr></table>'
def display_side_by_side(l, r):
    s = "display: inline-block;"
    html = to_html_table(l, style=s) + ' ' + to_html_table(r, style=s)
    display_html(HTML(data=html))

MVDs
===

MVDs.  Wow.  Everyone seems to be talking about them these days.  Is this because they're on several problems in PS2 and we skipped going over them formally in lecture?  Or... because they might just be the next big thing?

Either way, this notebook is meant to go into a little more depth about them, as well as to respond to some specific questions raised on Piazza.

**Note that there are some nice viualizations in the lecture slides that we won't reproduce here, so please look those over as well!  Also check out Activity-6-3...**

## Formal definition

Given a relation $R$ having attributes $A$, and two sets of attributes $X,Y\subseteq A$, the **_multi-value dependency (MVD)_** $X\twoheadrightarrow Y$ holds on $R$ if, for **any** tuples $t_1,t_2\in R$ s.t. $t_1[X] = t_2[X]$, there exists a tuple $t_3\in R$ s.t.:

* $t_3[X] = t_1[X] = t_2[X]$
* $t_3[Y] = t_1[Y]$
* $t_3[A\setminus Y] = t_2[A\setminus Y]$

where $A\setminus B$ = all elements of $A$ not in $B$.

So let's consider a toy example at this point:

In [8]:
%%sql
DROP TABLE IF EXISTS R; CREATE TABLE R (A INT, B INT, C INT);
INSERT INTO R VALUES (1, 1, 0);
INSERT INTO R VALUES (1, 0, 1);
SELECT * FROM R;

Done.
Done.
1 rows affected.
1 rows affected.
Done.


A,B,C
1,1,0
1,0,1


Let's consider the first two rows as $t_1$ and $t_2$ respectively; what is the $t_3$ that the definition tells us we must add if we want the MVD $\{A\}\twoheadrightarrow\{B\}$ to hold?  Let's add it in:

In [3]:
%sql INSERT INTO R VALUES (1,1,1); SELECT * FROM R;

1 rows affected.
Done.


A,B,C
1,1,0
1,0,1
1,1,1


However what about if we consider the first two rows as $t_2$ and $t_1$ respectively?  What row would the definition tell us we'd have to add in?

In [4]:
%sql INSERT INTO R VALUES (1,0,0); SELECT * FROM R;

1 rows affected.
Done.


A,B,C
1,1,0
1,0,1
1,1,1
1,0,0


Now, if we checked every other pair like this, we'd see that we are done- the MVD $\{A\}\twoheadrightarrow\{B\}$ holds!

## A second definition

We see that this suggests another (equivalent) definition, which we'll phrase slightly less formally:

**For an MVD $\{A\}\twoheadrightarrow\{B\}$ to hold, for any pair of tuples that agree on attributes $A$, we also must find the corresponding _"swapped"_ pair: a pair of tuples that also have the same value of $A$ but have their $B$ and $(A\cup B)^C$ attributes swapped.**

(*where $(A\cup B)^C$ just means the attributes not in $A$ or $B$*)

This definition should make sense, and might even feel more intuitive, given the toy example above!

### Addressing one point of confusion:

Remember, an MVD holds over a **relation** (not a single tuple, not a single pair of tuples $t_1,t_2$...).  Again, look at the definition- an MVD, say $\{X\}\twoheadrightarrow\{Y\}$, holds over a relation $R$ if **_for any possible_** pair of tuples $t_1,t_2$ in $R$ such that $t_1[A]=t_2[A]$, the condition in the definition holds...


### Another clarification of terminology:
So how can we ever _test_ if an MVD holds over a relation $R$, based just on one _instance_ of $R$?  Couldn't someone always add in more tuples and violate it?  Aren't we being a little sloppy when we say an MVD 'holds' on an _instance_ of a relation just based on checking the instance..?

This is correct.  Recall that we have the same situation with FDs- we need to have _external information_ (such as a set of functional dependencies) to _prove_ that an MVD holds in general over a relation.  We **can** prove that it is violated or not violated by an instance of $R$, and that it thus could or could not potentially hold in general.

So when we say an MVD (or an FD) _holds_ on an instance, based only on checking the instance, we implictly mean the above- that based on the instance we see, it _could_ hold in general.

### A third definition...

At this point (especially if you're still reading) you may suspect a connection to cross-products and joins.  Consider decomposing our toy example in its **original state (before we added in the two rows to make the MVD hold)**.  Let's decompose it into two tables, split by the attribute sets of the MVD (recall it was $\{A\}\twoheadrightarrow\{B\}$):

In [11]:
%%sql
DROP TABLE IF EXISTS R; CREATE TABLE R (A INT, B INT, C INT);
INSERT INTO R VALUES (1, 1, 0);
INSERT INTO R VALUES (1, 0, 1);
SELECT * FROM R;

Done.
Done.
1 rows affected.
1 rows affected.
Done.


A,B,C
1,1,0
1,0,1


In [10]:
%sql DROP TABLE IF EXISTS R1; CREATE TABLE R1 AS SELECT A,B FROM R GROUP BY A,B;
%sql DROP TABLE IF EXISTS R2; CREATE TABLE R2 AS SELECT A,C FROM R GROUP BY A,C;
r1 = %sql SELECT * FROM R1;
r2 = %sql SELECT * FROM R2;
display_side_by_side(r1,r2)

Done.
Done.
Done.
Done.
Done.
Done.


A B 1 0 1 1 A C 1 0 1 1

Now let's take the join to recompose:

In [13]:
%%sql
SELECT r1.A AS A, r1.B AS B, r2.C AS C
FROM R1 r1, R2 r2
WHERE r1.A = r2.A;

Done.


A,B,C
1,0,0
1,0,1
1,1,0
1,1,1


Woah!  The MVD that we decomposed along holds for the join of the decomposition!  We see that this is another definition for an MVD, in terms of the _'local joins'_: an MVD $\{A\}\twoheadrightarrow\{B\}$ holds if whenever we take a block of rows with the same value for the $A$ attributes, decompose it into $R_1(A,B)$ and $R_2(A,(A\cup B)^C)$, and then take the join of these tables, we end up with the same rows we had before!